## Relation Extraction

In [104]:
def relation_naming(label1,label2):
  if label1=='question' and label2=='answer':
    return 'parent'
  elif label1=='answer' and label2=='question':
    return 'child'
  elif label1=='header' and label2=='question':
    return 'parent'
  elif label1=='question' and label2=='header':
    return 'child'

## Import the scene graph file

In [105]:
!pwd

/Users/shashanksahoo/Documents/KIT_Projects_2022/ICA_project_2023/kit-docgcn-dla-service


In [106]:
# downloading the scene graph file
import json
with open('data/funsd_full_feature_sg_train_binary_0.json', 'r') as f:
    train_list_dict=json.load(f)
with open('data/funsd_full_feature_sg_test_binary_0.json', 'r') as f:
    eval_list_dict=json.load(f)

In [107]:
# train_list_dict

In [108]:
train_list_dict['0000971160']

{'file_name': '0000971160',
 'width': 762,
 'height': 1000,
 'objects': {'0': {'id': 3759,
   'box': [292, 91, 376, 175],
   'category': 'other',
   'text': 'R&D',
   'relations': {}},
  '1': {'id': 3760,
   'box': [219, 316, 225, 327],
   'category': 'question',
   'text': ':',
   'relations': {}},
  '2': {'id': 3761,
   'box': [95, 355, 169, 370],
   'category': 'question',
   'text': 'Suggestion:',
   'relations': {'0': {'name': 'answer', 'id': 13031, 'object': 3775}}},
  '3': {'id': 3762,
   'box': [482, 268, 518, 282],
   'category': 'question',
   'text': 'Date:',
   'relations': {'0': {'name': 'answer', 'id': 13032, 'object': 3771}}},
  '4': {'id': 3763,
   'box': [511, 309, 570, 323],
   'category': 'answer',
   'text': 'Licensee',
   'relations': {'0': {'name': 'question', 'id': 13033, 'object': 3772}}},
  '5': {'id': 3764,
   'box': [211, 651, 217, 662],
   'category': 'question',
   'text': '',
   'relations': {}},
  '6': {'id': 3765,
   'box': [461, 605, 483, 619],
   'cate

In [109]:
import pickle
# with open('funsd/textual_feature/funsd_train_bert_cls.pkl','rb') as f:
#     train_features = pickle.load(f)
  
with open('funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_train_parsing1_parsing2.pkl','rb') as f:
    train_features = pickle.load(f)
    

In [110]:
len(train_features)

7411

In [111]:
import pickle
# with open('funsd/textual_feature/funsd_test_bert_cls.pkl','rb') as f:
#     test_features = pickle.load(f)
 
with open('funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_test_parsing1_parsing2.pkl','rb') as f:
    test_features = pickle.load(f)
    
    

In [112]:
len(test_features)

2332

##Import Some External Features

## Define some pre-processing function

In [113]:
# according to id to search text
bbox_id_text = {}
for l in train_list_dict:
  for obj in train_list_dict[l]['objects']:
    id = train_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = train_list_dict[l]['objects'][obj]['text']


In [114]:
for l in eval_list_dict:
  for obj in eval_list_dict[l]['objects']:
    id = eval_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = eval_list_dict[l]['objects'][obj]['text']

In [115]:
# according to id to search object id in a document
def globalid_to_localid(id):
  for l in train_list_dict:
    for obj in train_list_dict[l]['objects']:
      if id == train_list_dict[l]['objects'][obj]['id']:
        return obj
  for l in eval_list_dict:
    for obj in eval_list_dict[l]['objects']:
      if id == eval_list_dict[l]['objects'][obj]['id']:
        return obj

In [116]:
train_list_dict.keys()

dict_keys(['92091873', '91939637', '87533049', '01073843', '92586242', '0012529284', '71341634', '0001477983', '91315069_91315070', '0060207528', '91161344_91161347', '71366499', '00922237', '91355841', '93380187', '01150773_01150774', '0060165115', '91914407', '92081358_1359', '660978', '0011974919', '81749056_9057', '00093726', '0000999294', '81186212', '0060077689', '87672097', '11875011', '00851772_1780', '0011859695', '81619511_9513', '0001456787', '81310636', '0060080406', '0011973451', '91104867', '00836244', '0001476912', '12052385', '0060255888', '71206427', '0012199830', '0001463448', '0000990274', '716552', '00920222', '0012529295', '80728670', '93213298', '71108371', '0001209043', '88057519', '01191071_1072', '00836816', '91361993', '92298125', '00040534', '0011838621', '0060029036', '0000989556', '0060024314', '0001129658', '0071032790', '00838511_00838525', '87682908', '0060214859', '00070353', '93351929_93351931', '0001438955', '81619486_9488', '00920294', '0000971160', 

In [117]:
i = 0
id_density_dict = {}
import numpy as np
for l in train_list_dict:
  id_density_dict[l] = {}
  for obj in train_list_dict[l]['objects']:
    id_density_dict[l][obj] = train_list_dict[l]['objects'][obj]['id']
    i+=1
# this dictionary used to transfer object id to text_density/text_number/char_density/char_number/visual_embedding/
i = 0
for l in eval_list_dict:
  id_density_dict[l] = {}
  for obj in eval_list_dict[l]['objects']:
    id_density_dict[l][obj] = eval_list_dict[l]['objects'][obj]['id']
    i+=1

In [118]:
import math
import torch
# one dimensional feature embedding
def positionalencoding1d(d_model, feature_list):
    """
    :param d_model: dimension of the model
    :param feature_list: length of positions
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(len(feature_list), d_model)
    feats = torch.tensor(feature_list).unsqueeze(1)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(math.log(10000.0) / d_model)))
    pe[:, 0::2] = torch.sin(feats.float() * div_term)
    pe[:, 1::2] = torch.cos(feats.float() * div_term)
    pe = pe.tolist()
    return pe


## Extracting Information from training and evaluation json file

In [119]:
# Generate local graph based training dataset objects and relations list
new_train_list_dict = {}
for l in train_list_dict:
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []
  
  for obj in train_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    for rel in train_list_dict[l]['objects'][obj]['relations']:
      tem_rel = [obj]
      obj2_id = train_list_dict[l]['objects'][obj]['relations'][rel]['object']
      obj2_id= globalid_to_localid(obj2_id)
      tem_rel.append(obj2_id)
      label1 = train_list_dict[l]['objects'][obj]['category']
      label2 = train_list_dict[l]['objects'][obj2_id]['category']
      rel_name = relation_naming(label1,label2)
      tem_rel.append(rel_name)
      tem_dic['relationships'].append(tem_rel)
  new_train_list_dict[l] = tem_dic

In [120]:
len(new_eval_list_dict)

50

In [121]:
# Generate local graph based validation dataset objects and relations list
new_eval_list_dict = {}
for l in eval_list_dict:
  print(l)
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []

  for obj in eval_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    for rel in eval_list_dict[l]['objects'][obj]['relations']:
      tem_rel = [obj]
      obj2_id = eval_list_dict[l]['objects'][obj]['relations'][rel]['object']
      obj2_id= globalid_to_localid(obj2_id)
      tem_rel.append(obj2_id)
      label1 = eval_list_dict[l]['objects'][obj]['category']
      label2 = eval_list_dict[l]['objects'][obj2_id]['category']
      rel_name = relation_naming(label1,label2)
      tem_rel.append(rel_name)
      tem_dic['relationships'].append(tem_rel)
  new_eval_list_dict[l] = tem_dic

82837252
85201976
86263525
82251504
93106788
82573104
87528321
85240939
82562350
82253245_3247
87093315_87093318
83443897
87332450
89856243
83635935
82250337_0338
82200067_0069
92380595
86236474_6476
82491256
85540866
86244113
83823750
83594639
87528380
86220490
87086073
87147607
82252956_2958
83641919_1921
82253058_3059
87125460
83624198
82504862
86328049_8050
82254765
86079776_9777
83553333_3334
83573282
87137840
87428306
86230203_0206
87594142_87594144
91814768_91814769
86075409_5410
83772145
82253362_3364
85629964
82092117
83996357


In [123]:
print(train_list_dict['92039708_9710']['objects']['4'])
print(train_list_dict['92039708_9710']['objects']['1']['bert_large_emb'])
print(new_train_list_dict['92039708_9710']['relationships'])

{'id': 4769, 'box': [487, 119, 624, 133], 'category': 'header', 'text': 'SUBMISSION DATE', 'relations': {'0': {'name': 'question', 'id': 14065, 'object': 4770}, '1': {'name': 'question', 'id': 14066, 'object': 4771}, '2': {'name': 'question', 'id': 14067, 'object': 4772}, '3': {'name': 'question', 'id': 14068, 'object': 4773}}}


KeyError: 'bert_large_emb'

In [124]:
#Using for searching real node size
num_obj_dict = {}
for l in new_train_list_dict:
    num_obj_dict[l] = len(new_train_list_dict[l]['objects'])

In [125]:
for l in new_eval_list_dict:
    num_obj_dict[l] = len(new_eval_list_dict[l]['objects'])

In [126]:
print(num_obj_dict)

{'92091873': 71, '91939637': 41, '87533049': 45, '01073843': 104, '92586242': 33, '0012529284': 71, '71341634': 104, '0001477983': 65, '91315069_91315070': 26, '0060207528': 44, '91161344_91161347': 148, '71366499': 34, '00922237': 35, '91355841': 34, '93380187': 70, '01150773_01150774': 33, '0060165115': 36, '91914407': 30, '92081358_1359': 54, '660978': 30, '0011974919': 42, '81749056_9057': 77, '00093726': 92, '0000999294': 181, '81186212': 11, '0060077689': 66, '87672097': 82, '11875011': 30, '00851772_1780': 33, '0011859695': 29, '81619511_9513': 82, '0001456787': 74, '81310636': 28, '0060080406': 59, '0011973451': 49, '91104867': 56, '00836244': 93, '0001476912': 52, '12052385': 20, '0060255888': 33, '71206427': 31, '0012199830': 27, '0001463448': 42, '0000990274': 31, '716552': 68, '00920222': 44, '0012529295': 88, '80728670': 36, '93213298': 67, '71108371': 82, '0001209043': 72, '88057519': 17, '01191071_1072': 24, '00836816': 30, '91361993': 40, '92298125': 20, '00040534': 81,

In [127]:
# only rename
list_dict_train={}
for l in new_train_list_dict:
    list_dict_train[l] = new_train_list_dict[l]

In [128]:
list_dict_test = {}
for l in new_eval_list_dict:
    list_dict_test[l] = new_eval_list_dict[l]

## Training and validation dataframe generation

In [129]:
#generate the training and labeling information for each image
#this information can be used to transfer object id to corresponding visual, text density or other kinds of features.
density_list_train = []
label_list_train = []
img_list_train = []
label_dict_train = {}
for l in train_list_dict:
  label_dict_train[l] = []
  for obj in train_list_dict[l]['objects']:
    density_list_train.append(obj)
    label_list_train.append(train_list_dict[l]['objects'][obj]['category'])
    img_list_train.append(l)
    label_dict_train[l].append(train_list_dict[l]['objects'][obj]['category'])
  #padding
  for obj in range(num_obj_dict[l],181):
    density_list_train.append(str(obj))
    label_list_train.append(str(-1))
    img_list_train.append(l)
    label_dict_train[l].append(str(-1))

In [130]:
# generating evalutaion labeling informaiton dictionary
density_list_eval = []
label_list_eval = []
label_dict_eval = {}
img_list_eval = []
for l in eval_list_dict:
  label_dict_eval[l] = []
  for obj in eval_list_dict[l]['objects']:
    density_list_eval.append(obj)
    label_list_eval.append(eval_list_dict[l]['objects'][obj]['category'])
    img_list_eval.append(l)
    label_dict_eval[l].append(eval_list_dict[l]['objects'][obj]['category'])
  for obj in range(num_obj_dict[l],181):
    density_list_eval.append(str(obj))
    label_list_eval.append(str(-1))
    img_list_eval.append(l)
    label_dict_eval[l].append(str(-1))

In [135]:
len(bert_list_train), len(bert_list_train[0])

(7411, 768)

In [136]:
len(bert_list_eval), len(bert_list_eval[0])

NameError: name 'bert_list_eval' is not defined

In [137]:
from pandas import DataFrame
df_train = DataFrame(density_list_train,columns=['density'])
df_train['label'] = label_list_train
df_train['image'] = img_list_train

In [138]:
df_eval = DataFrame(density_list_eval,columns=['density'])
df_eval['label'] = label_list_eval
df_eval['image'] = img_list_eval

In [139]:
label_list_train = df_train['label'].fillna('other').tolist()


In [140]:
label_list_eval = df_eval['label'].fillna('other').tolist()

In [142]:
label_list_train, label_list_eval

(['question',
  'question',
  'header',
  'question',
  'question',
  'answer',
  'answer',
  'answer',
  'answer',
  'answer',
  'answer',
  'question',
  'question',
  'question',
  'header',
  'question',
  'question',
  'question',
  'answer',
  'answer',
  'answer',
  'answer',
  'answer',
  'answer',
  'question',
  'question',
  'question',
  'other',
  'other',
  'answer',
  'answer',
  'answer',
  'other',
  'question',
  'other',
  'answer',
  'answer',
  'answer',
  'header',
  'answer',
  'answer',
  'answer',
  'question',
  'answer',
  'question',
  'question',
  'answer',
  'answer',
  'header',
  'question',
  'question',
  'question',
  'question',
  'question',
  'answer',
  'answer',
  'header',
  'question',
  'question',
  'question',
  'question',
  'answer',
  'answer',
  'answer',
  'answer',
  'question',
  'answer',
  'header',
  'question',
  'answer',
  'answer',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',
  '-1',


In [143]:
print(len(label_list_train)/181)

149.0


In [144]:
df_train_clean = df_train[df_train['label'].notnull()][['density','label','image']]

In [145]:
df_eval_clean = df_eval[df_eval['label'].notnull()][['density','label','image']] 

In [146]:
df_train_clean.head(20)

,density,label,image
0,0,question,92091873
1,1,question,92091873
2,2,header,92091873
3,3,question,92091873
4,4,question,92091873
5,5,answer,92091873
6,6,answer,92091873
7,7,answer,92091873
8,8,answer,92091873
9,9,answer,92091873


In [42]:
print(list_dict_train)

{'92091873': {'objects': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70'], 'relationships': [['0', '39', 'parent'], ['1', '40', 'parent'], ['2', '11', 'parent'], ['2', '51', 'parent'], ['2', '52', 'parent'], ['3', '48', 'child'], ['3', '5', 'parent'], ['4', '48', 'child'], ['4', '8', 'parent'], ['5', '3', 'child'], ['6', '49', 'child'], ['7', '50', 'child'], ['8', '4', 'child'], ['9', '11', 'child'], ['10', '52', 'child'], ['11', '2', 'child'], ['11', '9', 'parent'], ['12', '35', 'parent'], ['12', '36', 'parent'], ['12', '37', 'parent'], ['13', '35', 'parent'], ['13', '36', 'parent'], ['13', '37', 'parent'], ['14', '57', 'pa

In [147]:
label_list_train = df_train_clean['label'].to_list()
id_list_train = df_train_clean['density'].to_list()
img_list_train = df_train_clean['image'].to_list()

In [148]:
label_list_eval = df_eval_clean['label'].to_list()

In [149]:
id_list_eval = df_eval_clean['density'].to_list()

In [150]:
img_list_eval = df_eval_clean['image'].to_list()

In [151]:
obj_list_train = []
for i in range(len(id_list_train)):
  tem = []
  tem = [id_list_train[i],img_list_train[i]]
  obj_list_train.append(tem)


In [152]:
obj_list_eval = []
for i in range(len(id_list_eval)):
  tem = []
  tem = [id_list_eval[i],img_list_eval[i]]
  obj_list_eval.append(tem)

In [155]:
print(len(obj_list_train), len(obj_list_eval))

26969 9050


## Convert label into one-hot

In [156]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
num_class = len(list(set(label_list_train)))
lEnc = LabelEncoder()
lEnc.fit(np.unique(list(set(label_list_train))))
labels_one_hot_train = {}
for f in label_dict_train:
  num_labels = lEnc.transform(label_dict_train[f])
  labels_one_hot_train[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_train[f].append(to_add)


In [158]:
labels_one_hot_eval = {}
for f in label_dict_eval:
  num_labels = lEnc.transform(label_dict_eval[f])
  labels_one_hot_eval[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_eval[f].append(to_add)

In [159]:
print(len(labels_one_hot_train))

149


## Build Graph

In [53]:
print(new_train_list_dict.keys())

dict_keys(['92091873', '91939637', '87533049', '01073843', '92586242', '0012529284', '71341634', '0001477983', '91315069_91315070', '0060207528', '91161344_91161347', '71366499', '00922237', '91355841', '93380187', '01150773_01150774', '0060165115', '91914407', '92081358_1359', '660978', '0011974919', '81749056_9057', '00093726', '0000999294', '81186212', '0060077689', '87672097', '11875011', '00851772_1780', '0011859695', '81619511_9513', '0001456787', '81310636', '0060080406', '0011973451', '91104867', '00836244', '0001476912', '12052385', '0060255888', '71206427', '0012199830', '0001463448', '0000990274', '716552', '00920222', '0012529295', '80728670', '93213298', '71108371', '0001209043', '88057519', '01191071_1072', '00836816', '91361993', '92298125', '00040534', '0011838621', '0060029036', '0000989556', '0060024314', '0001129658', '0071032790', '00838511_00838525', '87682908', '0060214859', '00070353', '93351929_93351931', '0001438955', '81619486_9488', '00920294', '0000971160', 

In [160]:
node_lists_train = {}
for l in new_train_list_dict:
  node_lists_train[l] = []
  for obj in new_train_list_dict[l]['objects']:
    node_lists_train[l].append(obj)



In [161]:
node_lists_eval = {}
for l in new_eval_list_dict:
  node_lists_eval[l] = []
  for obj in new_eval_list_dict[l]['objects']:
    node_lists_eval[l].append(obj)

In [57]:
print(node_lists_train)

{'92091873': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70'], '91939637': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40'], '87533049': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44'], '01073843': ['0', '1', '2', '3', '4', '5', '6

In [162]:
# build local graph
object_graph_dict_train = {}
for l in new_train_list_dict:
  object_graph_dict_train[l] = {}
  for obj in new_train_list_dict[l]['objects']:
    object_graph_dict_train[l][obj] = {}
    object_graph_dict_train[l][obj]['obj2'] = []



In [163]:
object_graph_dict_eval = {}
for l in new_eval_list_dict:
  object_graph_dict_eval[l] = {}
  for obj in new_eval_list_dict[l]['objects']:
    object_graph_dict_eval[l][obj] = {}
    object_graph_dict_eval[l][obj]['obj2'] = []

In [164]:
print(len(list_dict_train))

149


In [165]:
print(list_dict_test.keys())
print(list_dict_test['83594639'])

dict_keys(['82837252', '85201976', '86263525', '82251504', '93106788', '82573104', '87528321', '85240939', '82562350', '82253245_3247', '87093315_87093318', '83443897', '87332450', '89856243', '83635935', '82250337_0338', '82200067_0069', '92380595', '86236474_6476', '82491256', '85540866', '86244113', '83823750', '83594639', '87528380', '86220490', '87086073', '87147607', '82252956_2958', '83641919_1921', '82253058_3059', '87125460', '83624198', '82504862', '86328049_8050', '82254765', '86079776_9777', '83553333_3334', '83573282', '87137840', '87428306', '86230203_0206', '87594142_87594144', '91814768_91814769', '86075409_5410', '83772145', '82253362_3364', '85629964', '82092117', '83996357'])
{'objects': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'], 'relationships': [['0', '20', 'parent'], ['1', '13', 'parent'], ['2', '5', 'parent'], ['3', '16', 'parent'], ['4', '15', 'child'], ['5', '2', 'child'], ['13', '1', 'c

In [166]:
#fill the empty graph
for sg in list_dict_train:
    obj_list = list_dict_train[sg]['objects']
    for rel_item in list_dict_train[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_train[sg][rel_item[0]]['obj2'].append(obj2)



In [167]:
#fill the empty graph
for sg in list_dict_test:
    obj_list = list_dict_test[sg]['objects']
    for rel_item in list_dict_test[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_eval[sg][rel_item[0]]['obj2'].append(obj2)

In [168]:
node_size_train = []
for f in object_graph_dict_train:
  l = object_graph_dict_train[f].keys()
  node_size_train.append(len(l))



In [169]:
node_size_eval = []
for f in object_graph_dict_eval:
  l = object_graph_dict_eval[f].keys()
  node_size_eval.append(len(l))

In [170]:

#padding methods
#weight for padding is -1
#node id if for padding nodes is -1
max_len = 177
col_train = {}
row_train = {}
weight_train = {}
for f in object_graph_dict_train:
  col_train[f] = []
  row_train[f] = []
  weight_train[f] = []
  for obj in object_graph_dict_train[f]:
    obj_rel_list = object_graph_dict_train[f][obj]['obj2']
    for obj2 in object_graph_dict_train[f]:
      if obj2 in obj_rel_list:
        weight_train[f].append(1.0)
      else:
        weight_train[f].append(0.0)
      row_train[f].append(obj)
      col_train[f].append(obj2)


In [171]:

#padding methods
#weight for padding is -1
#node id if for padding nodes is -1
max_len = 181
col_eval = {}
row_eval = {}
weight_eval = {}
for f in object_graph_dict_eval:
  col_eval[f] = []
  row_eval[f] = []
  weight_eval[f] = []
  for obj in object_graph_dict_eval[f]:
    obj_rel_list = object_graph_dict_eval[f][obj]['obj2']
    for obj2 in object_graph_dict_eval[f]:
      if obj2 in obj_rel_list:
        weight_eval[f].append(1.0)
      else:
        weight_eval[f].append(0.0)
      row_eval[f].append(obj)
      col_eval[f].append(obj2)

In [68]:
#PROBLEMS
# how to determine the weight to the node itself
# how to padding the node

In [172]:
import scipy.sparse as sp
node_size = 181
adj_train = {}
for f in weight_train:
  adj_train[f] = sp.csr_matrix((weight_train[f], (row_train[f], col_train[f])), shape=(node_size, node_size))

In [173]:
import scipy.sparse as sp
node_size = 181
adj_eval = {}
for f in weight_eval:
  adj_eval[f] = sp.csr_matrix((weight_eval[f], (row_eval[f], col_eval[f])), shape=(node_size, node_size))

##Build Model

In [174]:
# from inits import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res


class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])



class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)],
                              sparse=self.sparse_inputs)
            else:
                pre_sup = self.vars['weights_' + str(i)]            
            support = dot(self.support[i], pre_sup, sparse=True)
            supports.append(support)
        output = tf.add_n(supports)

        # bias
        if self.bias:
            output += self.vars['bias']
			
		
        self.embedding = output #output
        return self.act(output)


In [175]:
# from metrics import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)

In [176]:
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])
        self.pred = tf.argmax(self.outputs, 1)
        self.labels = tf.argmax(self.placeholders['labels'], 1)

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            featureless=False,
                                            sparse_inputs=True,
                                            logging=self.logging))
        
        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x, #
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [177]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    print(preds)
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))

    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

## Model Setting

In [178]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow.compat.v1 as tf

from sklearn import metrics
import random
import os
import sys


# # Set random seed
# seed = random.randint(1, 200)
# np.random.seed(seed)
# tf.set_random_seed(seed)


# Settings
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

flags = tf.app.flags
FLAGS = flags.FLAGS
for name in list(flags.FLAGS):
      delattr(flags.FLAGS,name)
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('dataset', 'sencegraph', 'Dataset string.')
# 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_string('model', 'gcn', 'Model string.')
flags.DEFINE_float('learning_rate', 0.00001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 100, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 768, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 0,
                   'Weight for L2 loss on embedding matrix.')  # 5e-4
flags.DEFINE_integer('early_stopping', 20,
                     'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

### Bert CLS as Node Feature

In [179]:
node_lists_train['92091873']

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70']

In [180]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*768)
    feature += id_density_dict[l][str(node)]
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),181):
    feature = np.array([0.0]*768)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])

In [181]:
import scipy.sparse as sp
features_eval = {}
for l in node_lists_eval:
  features_eval[l] = []
  for node in node_lists_eval[l]:
    feature = np.array([0.0]*768)
    feature += id_density_dict[l][str(node)]
    features_eval[l].append(feature)
  for i in range(len(node_lists_eval[l]),181):
    feature = np.array([0.0]*768)
    features_eval[l].append(feature)
  features_eval[l]=sp.csr_matrix(features_eval[l])

### Other features

In [182]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*768)
    feature += np.array((id_density_dict[l][str(node)]))
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),181):
    feature = np.array([0.0]*768)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])

In [80]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_eval = {}
for l in node_lists_eval:
  features_eval[l] = []
  for node in node_lists_eval[l]:
    feature = np.array([0.0]*768)
    feature += id_density_dict[l][str(node)]
    features_eval[l].append(feature)
  for i in range(len(node_lists_eval[l]),181):
    feature = np.array([0.0]*768)
    features_eval[l].append(feature)
  features_eval[l]=sp.csr_matrix(features_eval[l])

In [81]:
'''
import scipy.sparse as sp
features = []
for tokens in tokenize_nodes:
  feature = np.array([0.0]*2048)
  for token in tokens:
    try:
      feature += id_density_dict[int(tokens[0])]
    except:
      pass
  features.append(feature)
features= sp.csr_matrix(features)
'''

'\nimport scipy.sparse as sp\nfeatures = []\nfor tokens in tokenize_nodes:\n  feature = np.array([0.0]*2048)\n  for token in tokens:\n    try:\n      feature += id_density_dict[int(tokens[0])]\n    except:\n      pass\n  features.append(feature)\nfeatures= sp.csr_matrix(features)\n'

In [183]:
print(len(features_train))

149


### Sparse to tuple

In [184]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape
    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)
    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)
for l in features_train:
  features_train[l] = preprocess_features(features_train[l])
  
for l in features_eval:
  features_eval[l] = preprocess_features(features_eval[l])


/var/folders/rq/73_t64xn2zxf07vggynh53800000gq/T/ipykernel_3993/3828355148.py:21: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


## input setting

In [185]:
#define input dictionary
# num_obj_dict was defined to record the number of object in each document page
#
y_train = {}
train_mask = {}
for f in new_train_list_dict:
  y_train[f] = np.array([[0]*num_class]*adj_train[f].shape[0])
  y_train[f] = np.array(labels_one_hot_train[f])
  train_mask[f] = [False]*adj_train[f].shape[0]
  train_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]


In [186]:
y_val = {}
val_mask = {}
for f in new_eval_list_dict:
  y_val[f] = np.array([[0]*num_class]*adj_eval[f].shape[0])
  y_val[f] = np.array(labels_one_hot_eval[f])
  val_mask[f] = [False]*adj_eval[f].shape[0]
  val_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]

In [187]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)
support_train = {}
for f in adj_train:
  support_train[f] = [preprocess_adj(adj_train[f])]
support_eval = {}
for f in adj_eval:
  support_eval[f] = [preprocess_adj(adj_eval[f])]

In [188]:
# just change the number of support gpu
num_supports = 1
tf.compat.v1.disable_eager_execution()
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape = (5,512)),
    'labels': tf.placeholder(tf.float32, shape=(None, 5)),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0.5, shape=()),
    # helper variable for sparse dropout
    'num_features_nonzero': tf.placeholder(tf.int32)
}

# Create model
#print(features[2][1])
# placeholders: number of GCN

model = GCN(placeholders, input_dim=768, logging=True)

2023-05-08 16:14:53.050698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_15' with dtype int32
	 [[{{node Placeholder_15}}]]
2023-05-08 16:14:53.050819: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_15' with dtype int32
	 [[{{node Placeholder_15}}]]
2023-05-08 16:14:53.060977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_8' with dtype int64 and shape [?]
	 [[{{node Placeholder_8}}]]
2023-05-08

Tensor("graphconvolution_2_1/SparseTensorDenseMatMul/SparseTensorDenseMatMul:0", shape=(None, 5), dtype=float32)


2023-05-08 16:14:53.251733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_8' with dtype int64 and shape [?]
	 [[{{node Placeholder_8}}]]
2023-05-08 16:14:53.251839: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_8' with dtype int64 and shape [?]
	 [[{{node Placeholder_8}}]]


In [189]:
# Initialize session
session_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=session_conf)


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    feed_dict_val = construct_feed_dict(
        features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels, model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict_val)
    # print(outs_val[0], outs_val[1], outs_val[2], outs_val[3])
    return outs_val[0], outs_val[1], outs_val[2], outs_val[3], outs_val[4], outs_val[5]


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []




In [190]:
def construct_feed_dict(features, support, labels, labels_mask, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['support'][i]: support[i]
                      for i in range(len(support))})
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape})
    return feed_dict

## Training

In [191]:
# Train model
for epoch in range(FLAGS.epochs):
    
    # Construct feed dictionary
    if epoch == FLAGS.epochs-1:
        output_dic_train = {}
        output_dic_test = {}
    loss = 0
    for f in features_train:
        feed_dict = construct_feed_dict(
            features_train[f], support_train[f], y_train[f], train_mask[f], placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
        outs = sess.run([model.opt_op, model.loss, model.accuracy,
                         model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict)
        loss+=outs[1]
    print(loss)
print("Optimization Finished!")

239.14408147335052
237.93756532669067
236.44432318210602
234.7393273115158
232.87604546546936
230.8481124639511
228.67007517814636
226.35008823871613
223.91007506847382
221.4049048423767
218.85524010658264
216.29126155376434
213.7494456768036
211.29925858974457
208.8031325340271
206.49509990215302
204.26150596141815
202.1176519393921
200.2430064678192
198.35396122932434
196.64676237106323
195.17056000232697
193.69156634807587
192.40939843654633
191.2452014684677
190.29100906848907
189.32133078575134
188.41884994506836
187.69027245044708
186.97311007976532
186.39421558380127
185.87133133411407
185.20310711860657
184.95118939876556
184.56479740142822
184.22436833381653
183.80241739749908
183.37120509147644
183.2297726869583
182.94451773166656
182.69504404067993
182.4761511683464
182.24881690740585
181.92920035123825
181.78351366519928
181.9098760485649
181.64577454328537
181.24162405729294
181.15116649866104
180.99518138170242
180.9862706065178
180.75337040424347
180.5306704044342
180.57

In [192]:
for f in features_train:
    cost, acc, pred, labels, emb1, emb2 = evaluate(
        features_train[f], support_train[f], y_train[f], train_mask[f], placeholders)
    output_dic_train[f] = emb1

In [193]:
for f in features_eval:
    cost, acc, pred, labels, emb1, emb2 = evaluate(
        features_eval[f], support_eval[f], y_val[f], val_mask[f], placeholders)
    output_dic_test[f] = emb1

In [194]:
output_dic_train_bert = output_dic_train
output_dic_test_bert = output_dic_test

In [195]:
output_dic_train

{'92091873': array([[-0.00312843,  0.01389783, -0.00267059, ...,  0.00416981,
          0.01593803,  0.00030338],
        [-0.00562011,  0.02695449, -0.00695006, ...,  0.00840542,
          0.03538693,  0.00103364],
        [-0.00586547,  0.03120278, -0.00492332, ...,  0.00881278,
          0.03926197, -0.00082247],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 '91939637': array([[-0.00621742,  0.02678216, -0.00443824, ...,  0.00754959,
          0.03398317,  0.00211232],
        [-0.00298914,  0.02650988, -0.00550415, ...,  0.00652206,
          0.03335226, -0.00128523],
        [-0.00515226,  0.02826296, -0.00488682, ...,  0.0078617 ,
          0.03421407, -0.00294439],
        ...,
        [ 0.   

In [196]:
output_dic_test

{'82837252': array([[-0.00590837,  0.02893899, -0.0041767 , ...,  0.00895151,
          0.03572142, -0.00029742],
        [-0.00521453,  0.02664782, -0.00464847, ...,  0.00684891,
          0.0313712 ,  0.00121954],
        [-0.00638074,  0.02682901, -0.00425361, ...,  0.00721391,
          0.03351588,  0.00309443],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 '85201976': array([[-0.0040057 ,  0.02799834, -0.00465604, ...,  0.00901947,
          0.03539767,  0.00244797],
        [-0.00530785,  0.02835287, -0.00681311, ...,  0.01062309,
          0.03875764, -0.00079143],
        [-0.00739955,  0.02688012, -0.00370066, ...,  0.00554125,
          0.03588308, -0.00181207],
        ...,
        [ 0.   

In [197]:
for img in output_dic_train_bert:
  for i in range(len(train_list_dict[img]['objects'].keys())):
    train_list_dict[img]['objects'][str(i)]['gcn_bert_large'] = output_dic_train_bert[img][i]

for img in output_dic_test_bert:
  for i in range(len(eval_list_dict[img]['objects'].keys())):
    eval_list_dict[img]['objects'][str(i)]['gcn_bert_large'] = output_dic_test_bert[img][i]

In [198]:
bert_list_train = []
bert_list_test = []

In [199]:
for img in train_list_dict:
  for obj in train_list_dict[img]['objects']:
    c_obj = train_list_dict[img]['objects'][obj]
    bert_list_train.append(c_obj['gcn_bert_large'])

for img in eval_list_dict:
  for obj in eval_list_dict[img]['objects']:
    c_obj = eval_list_dict[img]['objects'][obj]
    bert_list_test.append(c_obj['gcn_bert_large'])

In [234]:
# train_list_dict['92091873']
# len(bert_list_train[0])

In [200]:
type(train_list_dict['92091873']['objects']['0']['gcn_bert_large'])

numpy.ndarray

In [240]:
!pip install numpyencoder

In [201]:
import json
from numpyencoder import NumpyEncoder


with open('funsd/funsd_train_list_dict.json', 'w') as fpw:
    json.dump(train_list_dict, fpw, cls=NumpyEncoder)

In [202]:
with open('funsd/funsd_test_list_dict.json', 'w') as fp:
    json.dump(eval_list_dict, fp, cls=NumpyEncoder)

In [205]:
import pickle
with open('funsd/funsd_bert_list_train.pkl','wb') as f:
  pickle.dump(bert_list_train,f)

In [203]:
len(bert_list_train), len(bert_list_train[0])

(7411, 768)

In [206]:
import pickle
with open('funsd/funsd_bert_list_test.pkl','wb') as f:
  pickle.dump(bert_list_test,f)

In [207]:
len(bert_list_test), len(bert_list_test[0])

(2332, 768)

In [204]:
df_train.columns

Index(['density', 'label', 'image'], dtype='object')